Врублевская Екатерина, 3 курс 13 группа

In [67]:
import pandas as pd
from numpy import NaN
import numpy as np
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [73]:
data = pd.read_csv('./train.csv')
print(data.corr())

data = data.drop(columns=['PassengerId', 'Name', 'SibSp','Ticket', 'Fare', 'Cabin', 'Embarked'])
data['Sex'] = data['Sex'].replace(to_replace='female', value=0).replace(to_replace='male', value=1)

_, test_data = train_test_split(data.dropna(), test_size=0.25)
train_data = data.drop(index=test_data.index)

x_test, y_test = test_data[['Pclass', 'Sex', 'Age', 'Parch']], test_data[['Survived']]
print(test_data)

cleared_data = train_data.dropna()
added_data = train_data

avg_year = added_data['Age'].sum() / added_data['Age'].count() 
print(avg_year)
added_data['Age'] = added_data['Age'].replace(to_replace=NaN, value=avg_year)

             PassengerId  Survived    Pclass       Age     SibSp     Parch  \
PassengerId     1.000000 -0.005007 -0.035144  0.036847 -0.057527 -0.001652   
Survived       -0.005007  1.000000 -0.338481 -0.077221 -0.035322  0.081629   
Pclass         -0.035144 -0.338481  1.000000 -0.369226  0.083081  0.018443   
Age             0.036847 -0.077221 -0.369226  1.000000 -0.308247 -0.189119   
SibSp          -0.057527 -0.035322  0.083081 -0.308247  1.000000  0.414838   
Parch          -0.001652  0.081629  0.018443 -0.189119  0.414838  1.000000   
Fare            0.012658  0.257307 -0.549500  0.096067  0.159651  0.216225   

                 Fare  
PassengerId  0.012658  
Survived     0.257307  
Pclass      -0.549500  
Age          0.096067  
SibSp        0.159651  
Parch        0.216225  
Fare         1.000000  
     Survived  Pclass  Sex   Age  Parch
418         0       2    1  30.0      0
872         0       1    1  33.0      0
278         0       3    1   7.0      1
194         1       1  

added_data - датасет где все NaN в столбце возраст заменены на среднее арифметическое всех значений столбца, 

cleared_data - очищенный от таких строк датасет (где есть NaN),

Обучаю алгоритмы на очищенном датасете:

In [74]:
x_cleared, y_cleared = cleared_data[['Pclass', 'Sex', 'Age', 'Parch']], cleared_data[['Survived']]
d_tree = DecisionTreeClassifier(
    criterion='gini',
    max_depth=4,
    min_samples_leaf=12
)
d_tree.fit(x_cleared, y_cleared)
y_predicted = d_tree.predict(x_test)
print("Tree:")
print(accuracy_score(y_test, y_predicted))
 
neigh = KNeighborsClassifier(n_neighbors=3, metric="minkowski")
neigh.fit(x_cleared, y_cleared['Survived'])
y_predicted = neigh.predict(x_test)
print("kNN:")
print(accuracy_score(y_test, y_predicted))

Tree:
0.7932960893854749
kNN:
0.7486033519553073


Нормализую признаки Age и Pclass c помощью MinMaxScaler и снова обучаю алгоритмы

In [75]:
norm_cleared_data = cleared_data.copy()

scaler = preprocessing.MinMaxScaler() 
arr_scaled = scaler.fit_transform(norm_cleared_data)
norm_cleared_data = pd.DataFrame(arr_scaled, columns=norm_cleared_data.columns,index=norm_cleared_data.index)

x_train, y_train = norm_cleared_data[['Pclass', 'Sex', 'Age', 'Parch']], norm_cleared_data[['Survived']]

d_tree = DecisionTreeClassifier(
    criterion='gini',
    max_depth=4,
    min_samples_leaf=12
)
d_tree.fit(x_train, y_train)
 
arr_scaled = scaler.fit_transform(x_test)
x_test_norm = pd.DataFrame(arr_scaled, columns=x_test.columns,index=x_test.index)

    
y_predicted = d_tree.predict(x_test_norm)
print("Tree:")
print(accuracy_score(y_test, y_predicted))

neigh = KNeighborsClassifier(n_neighbors=3, metric="minkowski")
neigh.fit(x_train, y_train['Survived'])

y_predicted = neigh.predict(x_test_norm)
print("kNN:")
print(accuracy_score(y_test, y_predicted))

Tree:
0.7932960893854749
kNN:
0.7988826815642458


далее запускаю на дополненном датасете

In [76]:
x_train, y_train = added_data[['Pclass', 'Sex', 'Age', 'Parch']], added_data[['Survived']]
print(added_data)
d_tree = DecisionTreeClassifier(
    criterion='gini',
    max_depth=4,
    min_samples_leaf=12
)
d_tree.fit(x_train, y_train)

y_predicted = d_tree.predict(x_test)
print("Tree:")
print(accuracy_score(y_test, y_predicted))
 
neigh = KNeighborsClassifier(n_neighbors=3, metric="minkowski")
neigh.fit(x_train, y_train['Survived'])
y_knn = neigh.predict(x_test)
print("kNN:")
print(accuracy_score(y_test, y_knn))

     Survived  Pclass  Sex        Age  Parch
1           1       1    0  38.000000      0
2           1       3    0  26.000000      0
3           1       1    0  35.000000      0
4           0       3    1  35.000000      0
5           0       3    1  29.681477      0
..        ...     ...  ...        ...    ...
884         0       3    1  25.000000      0
885         0       3    0  39.000000      5
887         1       1    0  19.000000      0
888         0       3    0  29.681477      2
889         1       1    1  26.000000      0

[712 rows x 5 columns]
Tree:
0.8100558659217877
kNN:
0.770949720670391


и на дополненном нормализованном датасете:

In [77]:
norm_added_data = added_data.copy()

scaler = preprocessing.MinMaxScaler() 
arr_scaled = scaler.fit_transform(norm_added_data)
norm_added_data = pd.DataFrame(arr_scaled, columns=norm_added_data.columns,index=norm_added_data.index)

x_train, y_train = norm_added_data[['Pclass', 'Sex', 'Age', 'Parch']], norm_added_data[['Survived']]

d_tree = DecisionTreeClassifier(
    criterion='gini',
    max_depth=4,
    min_samples_leaf=12
)
d_tree.fit(x_train, y_train)

arr_scaled = scaler.fit_transform(x_test)
x_test_norm = pd.DataFrame(arr_scaled, columns=x_test.columns,index=x_test.index)


y_predicted = d_tree.predict(x_test_norm)
print("Tree:")
print(accuracy_score(y_test, y_predicted))
 
neigh = KNeighborsClassifier(n_neighbors=3, metric="minkowski")
neigh.fit(x_train, y_train['Survived'])
y_predicted = neigh.predict(x_test_norm)
print("kNN:")
print(accuracy_score(y_test, y_predicted))

Tree:
0.8212290502793296
kNN:
0.7988826815642458


Вывод: на метрики дерева решений нормализация данных влияния практически не оказывает (хотя по логике не должна вообще оказывать)
Очищенный: 0.7932960893854749
Очищенный нормализованный: 0.7932960893854749
Заполненный: 0.8100558659217877
Заполненный нормализованный: 0.8212290502793296
А вот заполнение пропусков помогает немного улучшить метрику.
В случае с kNN и нормализация и заполнение пропусков помогают немного улучшить метрики:
Очищенный: 0.7486033519553073
Очищенный нормализованный: 0.7988826815642458
Заполненный: 0.770949720670391
Заполненный нормализованный: 0.7988826815642458